# 1. Venues collection notebook
----------------
-----------

## This notebook consolidates 3 tools: 
1. Web scraper [BeautifulSoup]
2. Geo coords downloader [Nominatim]
3. Venues info downloader [Foursquare]

---
---

## Table of contents <a name="contentzs"></a>
* [Scraping web pages](#web)
* [Fetching geo coords](#geo)
    * [Check boroughs overlap](#overlap)
    * [Mapping all boroughs](#map-boro)
* [Downloading venues info](#venues)
    * [Mapping venues in a city](#map-ven)

----------
----------

Load libraries

In [1]:
%%time
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
# import matplotlib.pyplot as plt
# import matplotlib.cm as cm
# import matplotlib.colors as colors
import folium # map rendering library
from folium.plugins import MarkerCluster  ######## for venue clusters
from bs4 import BeautifulSoup as bs
import fnmatch
import warnings

print('Libraries imported.')

Libraries imported.
Wall time: 1.71 s


------
------

Load Foursquare credentials

In [2]:
CLIENT_ID = 'TFS1KPHYLTVIWSSUYDZGOUIS3WYLE3J5KNKVA0Z2XDQKARRL' #'your-client-ID' # your Foursquare ID
CLIENT_SECRET = 'XAWSGVIWAWXETDOWEC51VYLK1U5HFGAMPQWB2NTWJCUKXMRV' #'your-client-secret' # your Foursquare Secret
VERSION = '20180605'  # Foursquare API version

# print('Your credentials:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

Quick test of foursquare url query

In [3]:
address = 'Municipio 1,Milano,Italy' 
geolocator = Nominatim(user_agent="exploramientu")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

cutoff = 20
print(f'{address[:min(cutoff,len(address))]}\
 geo coords are: Lat = {round(latitude,4)} , Lon = {round(longitude,4)}') 
print('¬¬¬¬'*20,'\n')

print(f'See below some venues for {address}.')
print(20*'----')
    
radius = 500
limit = 10

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, limit)

results = requests.get(url).json()
venues = results['response']['groups'][0]['items']

for i,v in enumerate(venues):
    print( i, v['venue']['name'], '-->',
#           v['venue']['location']['lat'],
#           v['venue']['location']['lng'],
          v['venue']['categories'][0]['name'] ) 

Municipio 1,Milano,I geo coords are: Lat = 45.4673 , Lon = 9.186
¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬ 

See below some venues for Municipio 1,Milano,Italy.
--------------------------------------------------------------------------------
0 Signorvino --> Wine Bar
1 Giovanni Cova & C. --> Bakery
2 Antonia --> Accessories Store
3 Starbucks Reserve Roastery --> Coffee Shop
4 Moleskine Store --> Paper / Office Supplies Store
5 Risoelatte --> Italian Restaurant
6 Venchi --> Ice Cream Shop
7 Flying Tiger --> Gift Shop
8 HMS Hotel Milano Scala --> Hotel
9 Princi --> Bakery


#### [back to contents](#contentzs)

------------
-------
----
-----

### Scrape city page on wikipedia to retrieve list of boroughs <a name="web"></a>

https://it.wikipedia.org/wiki/Municipi_di_Milano#Schema_delle_zone_di_Milano

In [70]:
url = 'https://it.wikipedia.org/wiki/Municipi_di_Milano#Schema_delle_zone_di_Milano'
# 'https://en.wikipedia.org/wiki/List_of_neighborhoods_in_San_Francisco#Alamo_Square'
# 'https://en.wikipedia.org/wiki/Abu_Dhabi#Neighborhoods'
# 'https://en.wikipedia.org/wiki/Planning_Areas_of_Singapore'
# 'https://en.wikipedia.org/wiki/Local_government_areas_of_New_South_Wales'
# 'https://en.wikipedia.org/wiki/Special_wards_of_Tokyo'
# 'https://en.wikipedia.org/wiki/Subdivisions_of_Zurich'
# 'https://en.wikipedia.org/wiki/Arrondissements_of_Paris'
# 'https://en.wikipedia.org/wiki/London_boroughs'
# 'https://it.wikipedia.org/wiki/Municipi_di_Milano#Schema_delle_zone_di_Milano'
req = requests.get(url)
soup = bs(req.content,'lxml')  

table = soup.find_all('table' , {'class': 'wikitable sortable'} ) #, headers=headers)
# also try: soup.find('table', {'class': 'wikitable sortable'}) 

# table = soup.find_all({'li', 'span'}, {'class':"toctext"})
# table = soup.find_all('span', {'class':"mw-headline", 'id':"Neighborhoods"})
# table[0].text
# finds = table.find_all({'li','a'}) #, {'href'})  #, {'class':'new'}) #:"Città Studi"})

# table = soup.find_all({'li':'a'})
# table[50].text
# nbs = []
# for i in range(0,118):   #len(table)):
# #     print(table[i+1].text)
#     nbs.append(table[i+1].text)
# nbs
    
# txt = table[0].text; txt
# table[1].find_all('td' , {'align':'left'})[0].text.strip()  #{'a class':'mw-redirect' }) #.text  
#="mw-redirect")[0]  #.find_all('class')  #="mw-redirect"')[0]


# # IF TABLE AVAILABLE
df = pd.read_html(str(table))
boros = pd.DataFrame(df[0])
boros.iloc[:5,:5]  #.head()

,#,Denominazione,Superficie(km²),Abitanti(31.12.2018),Densità(ab/km²)
0,Municipio 1,Centro storico,967,98 531,10 189
1,Municipio 2,"Stazione Centrale, Gorla, Turro, Greco, Cresce...",1258,162 090,12 884
2,Municipio 3,"Città Studi, Lambrate, Venezia",1423,144 110,10 127
3,Municipio 4,"Vittoria, Forlanini",2095,161 551,7 711
4,Municipio 5,"Vigentino, Chiaravalle, Gratosoglio",2987,126 089,4 221


#### Cleaning downloaded table

In [13]:
# clean-up
boros_clean = pd.DataFrame(data=nbs)
# pd.DataFrame(data=boros.loc[ boros['Region']=='Central' ]).iloc[:,[0]]
# boros_clean = pd.DataFrame(data=boros.iloc[:8,:1]) 
    # pd.DataFrame(data=boros.loc[ boros['Designation']=='Inner', 'London borough' ] )
boros_clean['City+'] = 'San Francisco, CA, United States'
boros_clean.columns = ['Borough', 'City+']

# overrides for PARIS
# num_top_boros = 11
# indicators = ['st', 'nd', 'rd']
# for b in range(num_top_boros):
#         try:
#             boros_clean.iloc[b,0] = '{}{} Arrondissement'.format(b+1, indicators[b])
#         except:
#             boros_clean.iloc[b,0] = ('{}th Arrondissement'.format(b+1)) 
    
## GENERIC OVERRIDES ##
# num_top_boros = 8
# for b in range(num_top_boros):
#     boros_clean.iloc[b,0] = 'District {}'.format(b+1) #, indicators[b])

# overrides for ZURICH
# boros_clean.iloc[0,0] = 'Altstadt'  #District 1
# boros_clean.iloc[1,0] = 'Wollishofen'  #District 2
# boros_clean.iloc[2,0] = 'Wiedikon'  #District 3
# boros_clean.iloc[3,0] = 'Aussersihl'  #'District 4 
# boros_clean.iloc[4,0] = 'Industriequartier' #District 5
# boros_clean.iloc[5,0] = 'Kirche Unterstrass'  #District 6
# boros_clean.iloc[6,0] = 'Hottingen'   #District 7
# boros_clean.iloc[7,0] = 'Riesbach'  #District 8

# overrides for lONDON [very asymmetrical boroughs]
# boros_clean['Borough'][3] = boros_clean['Borough'][3].strip('[notes 2]')
# boros_clean.iloc[6,0] = 'Brixton'  # centre for borough Lambeth
# boros_clean.iloc[7,0] = 'Catford'  # centre for borough Lewisham
# boros_clean.iloc[8,0] = 'Borough'  #'Borough market' 'Guy's hospital # centre for borough Soutwark
# boros_clean.iloc[11,0] = 'Bond Street'  # centre for borough Westminster

# override for Singapore
# boros_clean.iloc[6,0] = 'Bond Street'
# boros_clean.dropna(inplace=True)
# boros_clean.to_frame().rename(columns={'#':'Borough'},inplace=True)
# boros_clean.reset_index(drop=True,inplace=True)
# boros_clean.drop(index=[0,2,4,9,12,15],inplace=True)
boros_clean.reset_index(drop=True,inplace=True)
boros_clean

,Borough,City+
0,Anza Vista,"San Francisco, CA, United States"
1,Ashbury Heights,"San Francisco, CA, United States"
2,Balboa Park,"San Francisco, CA, United States"
3,Balboa Terrace,"San Francisco, CA, United States"
4,Bayview,"San Francisco, CA, United States"
...,...,...
113,West Portal,"San Francisco, CA, United States"
114,Western Addition,"San Francisco, CA, United States"
115,Westwood Highlands,"San Francisco, CA, United States"
116,Westwood Park,"San Francisco, CA, United States"


In [51]:
# boros_clean
# str('Canada Bay, City of').split(',')[0]

temp file csv to append data

In [14]:
# first save (and create file)
# boros_clean.to_csv('boroughs_list.csv',index=False)

In [126]:
# check file before append
# pd.read_csv('boroughs_list.csv')  #.tail()

In [107]:
# subsequent saves --> append mode
# boros_clean.to_csv('boroughs_list.csv', mode='a', index=False , header=None)

In [74]:
# check file after append
# pd.read_csv('boroughs_list.csv')

In [97]:
# # pd.ExcelWriter either writes in separate sheets or overrides target sheet....

# writer = pd.ExcelWriter('test2.xlsx', engine='openpyxl')
# boros_milan.to_excel(writer, index=False)
# writer.save()

# ldf = len(pd.read_excel('test2.xlsx'))
# appender = pd.ExcelWriter('test2.xlsx', engine='openpyxl' , mode='a')
# boros_milan.to_excel(appender, sheet_name='Sheet1', \
#                      index=False , startrow=ldf+1 , header=None)
# appender.save()

#### [back to contents](#contentzs)

---------
----

### Fetch **geo coords** for each borough <a name="geo"></a>

In [5]:
# default import from above web scraping process
# locs = boros_clean

#
# NB if imported from file --> ALL boroughs from all cities!!!
# locs = pd.read_csv('boroughs_list.csv')  #NB RAW LIST
#
# locs = pd.read_csv('hoods_list_SANFRANC-TO BE FOUND.csv')
# # OR import file and then apply loc selection
lox = pd.read_csv('boroughs_list.csv')

bl = list( lox['City+'].unique() )
for i,n in enumerate(bl):
    print(i,n) 
print('\n')

ci = bl[9]  # enter city index number here
locs = lox.loc[ lox['City+']==ci ]
# # # locs['City+'].iloc[0]  #.iloc[0,0]
locs.tail()

0 Zurich, Switzerland
1 Milan, Italy
2 New York, NY, United States
3 Dubai, United Arab Emirates
4 Tokyo, Japan
5 London, United Kingdom
6 Singapore, Singapore
7 San Francisco, CA, United States
8 Paris, France
9 Sydney, Australia




,Borough,City+
145,Clovelly,"Sydney, Australia"
146,Forest Lodge,"Sydney, Australia"
147,Lavender Bay,"Sydney, Australia"
148,Pyrmont,"Sydney, Australia"
149,Macdonaldtown,"Sydney, Australia"


In [6]:
%%time
geolocator = Nominatim(user_agent="exploramaunz")
lat=[];lon=[]
warnings.filterwarnings("ignore")

# for c in range(len(locs)):
for bo,ci in zip(locs['Borough'], locs['City+']):  
    try:
    #     print(bo+', '+ci)

        loc_c = geolocator.geocode(bo+', '+ci)
                #locs['Borough'].fillna('').iloc[c]+','+locs['City+'].iloc[c])  
            
        try:
            lat_c = loc_c.latitude                        
        except:
            lat_c = 'N/A'
            
        try: 
            lon_c = loc_c.longitude
        except:
            lon_c = 'N/A'
            
        lat.append(lat_c)
        lon.append(lon_c)
        
#         print(bo,':', lat_c,',', lon_c)

    #     print(c+1, locs['Borough'].fillna('')[c], locs['City'][c],\
    #           locs['Nation'][c], lat_c, lon_c)

    # if exceeded Nominatim's stated maximum rate of 1 request per second 
    # add this in the loop: sleep(1.1)                 
        sleep(1.1)
        
    except:
        continue
               

locs['Latitude'] = lat
locs['Longitude'] = lon

# warnings.filterwarnings("default")

locs.tail()

Wall time: 6.58 s


,Borough,City+,Latitude,Longitude
145,Clovelly,"Sydney, Australia",-33.912639,151.261790
146,Forest Lodge,"Sydney, Australia",-33.880556,151.178333
147,Lavender Bay,"Sydney, Australia",-33.843200,151.207415
148,Pyrmont,"Sydney, Australia",-33.869214,151.192363
149,Macdonaldtown,"Sydney, Australia",-33.896905,151.185892


temp file csv to append data

In [ ]:
# lat

In [56]:
# first save (and create file)
# locs.to_csv('boroughs_list_with_coords.csv',index=False)

In [76]:
# check file before append
# pd.read_csv('boroughs_list_with_coords.csv').tail(20)

In [135]:
# subsequent saves --> append mode
# locs.to_csv('boroughs_list_with_coords.csv', mode='a', index=False , header=None)

In [137]:
# check file after append
# pd.read_csv('boroughs_list_with_coords.csv').tail(20)

#### [back to contents](#contentzs)

----
----

#### Map locations with **radius** in a city to check overlapping <a name="overlap"></a>

In [7]:
# load from file
checks = pd.read_csv('boroughs_list_TO_CHECK.csv')  #_SELECTFEW.csv')
# checks['City+'][0]
checks.tail()  #.iloc[35:45,:] 

,Borough,City+,Latitude,Longitude
9,Clovelly,"Sydney, Australia",-33.912639,151.261790
10,Forest Lodge,"Sydney, Australia",-33.880556,151.178333
11,Lavender Bay,"Sydney, Australia",-33.843200,151.207415
12,Pyrmont,"Sydney, Australia",-33.867555,151.192691
13,Macdonaldtown,"Sydney, Australia",-33.896783,151.186337


In [8]:
%%time
# centre the map on chosen city
city0 = checks['City+'][0]
geolocator = Nominatim(user_agent="expla")
loc0 = geolocator.geocode(city0)
lat0 = loc0.latitude
lon0 = loc0.longitude

boris=[];latis=[];longis=[]
# create map using latitude and longitude values
map_checks  = folium.Map(location=[lat0, lon0], zoom_start=12)
                        #tiles='Stamen Toner')

for bo, ci, lati, long in zip(checks['Borough'], checks['City+'],
                           checks['Latitude'], checks['Longitude']):
    try:
        
        loc_bo = geolocator.geocode(bo+','+ci)

        # if lat/lon not available, find it
        if pd.isna(lati)==False:
            lat_bo = lati
        else:
            lat_bo = loc_bo.latitude

        if pd.isna(long)==False:
            lon_bo = long
        else:
            lon_bo = loc_bo.longitude

        boris.append(bo)
        latis.append(lat_bo)
        longis.append(lon_bo)

        label = '{}, {}'.format(bo, ci)
        label = folium.Popup(label, parse_html=True)
        folium.Circle(  #CircleMarker  ## use Circle to show radius in meters
            [lat_bo, lon_bo],
            radius=1000,
            popup=label,
            color='red',
            weight=2,
            fill=True,
            fill_color='red',  #'#3186cc',
            fill_opacity=0.2,
            parse_html=False).add_to(map_checks)
    
    except:
        continue

#     print(bo,':', lat_bo,',', lon_bo)
# print('\n')
map_checks

Wall time: 7.11 s


In [ ]:
# map_checks.save(outfile= "sf_neighborhoods_selectASFDASFFZXC.html")

In [20]:
coords = pd.DataFrame(data={'Area':boris,'Lat':latis,'Lon':longis}) 
# coords #.shape #.iloc[35:45,:]

#### [back to contents](#contentzs)

----
-----

#### Visualize **map** of all boroughs <a name="map-boro"></a>

In [9]:
# all_locs = locs #temp
all_locs = pd.read_csv('boroughs_list_with_coords.csv')
all_locs.tail(15)

,Borough,City+,Latitude,Longitude
135,Arc de Triomphe,"Paris, France",48.873779,2.295037
136,City {*},"Sydney, Australia",-33.865423,151.207317
137,Woollahra - Double Bay,"Sydney, Australia",-33.880873,151.243412
138,Bondi,"Sydney, Australia",-33.893056,151.263333
139,Surry Hills - Darlinghurst,"Sydney, Australia",-33.880456,151.216719
140,Elizabeth Bay House,"Sydney, Australia",-33.870085,151.226444
141,Randwick,"Sydney, Australia",-33.914121,151.241005
142,Rozelle,"Sydney, Australia",-33.864500,151.174354
143,Waterloo,"Sydney, Australia",-33.900276,151.207314
144,Chippendale - Ultimo,"Sydney, Australia",-33.884304,151.200078


In [11]:
# address_0 = 'Frankfurt,Germany' 
# geolocator = Nominatim(user_agent="exploratur")
# location_0 =  geolocator.geocode(address_0)
latitude_0 = 45.5  #location_0.latitude
longitude_0 = 9.2  #location_0.longitude

map  = folium.Map(location=[latitude_0, longitude_0], zoom_start=2) 
    # , titles=title, attr="attribution")


for lat, lon, city, bor in zip(all_locs['Latitude'], all_locs['Longitude'],\
                               all_locs['City+'], all_locs['Borough']):
    label = '{}, {}'.format(bor, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,  # for Circle this is radius in meters!
        popup=label,
        color='black',
        weight=2,
        fill=True,
        fill_color='red', #yellow',  #'#3186cc',
        fill_opacity=0.8,
        parse_html=False).add_to(map)

map

#### [back to contents](#contentzs)

----
-----

### Fetch **venues** for each borough <a name="venues"></a>

In [76]:
hoods = pd.read_csv('boroughs_list_with_coords.csv')
# hoods #.head()

# # for cities list:
v = list( hoods['City+'].unique() )
for i,n in enumerate(v):
    print(i,n) 
    
    
# # for one area only of city c use this:
# # set city first:
# cit = 'Tokyo, Japan'  # 'City+'
# v = list( hoods.loc[ hoods['City+']==cit,'Borough' ].unique() )
# for i,n in enumerate(v):
#     print(i,n)

0 Zurich, Switzerland
1 Milan, Italy
2 New York, NY, United States
3 Dubai, United Arab Emirates
4 Tokyo, Japan
5 London, United Kingdom
6 Singapore, Singapore
7 San Francisco, CA, United States
8 Paris, France
9 Sydney, Australia


In [77]:
# !!! limit foursquare fetch to one city per query to avoid hitting calls limits !!!

# # enter city/area index number here
ci = v[9] 
# print(ci ,'\n')

# # for all areas of a city use this
hoods_ci = hoods.loc[ hoods['City+']==ci ]
# # for one area only use this
# hoods_ci = hoods.loc[ hoods['Borough']==ci ]


# # MULTIPLE SELECTION
# list_hoods = [ 'Buena Vista Park {*}','Inner Sunset {*}','Jordan Park {*}' ] # 'Chelsea-Knightsbridge' ] 
# hoods_ci = hoods.loc[ hoods['Borough'].isin(list_hoods) ]


# # NEW AREA; MANUAL ENTRY
# hoods_ci = pd.DataFrame( { 'Borough': ['Invalides - La Tour-Maubourg'],
#            'City+': ['Paris, France'],
#             'Latitude':48.857950 ,
#             'Longitude': 2.310185  } )

hoods_ci

,Borough,City+,Latitude,Longitude
136,City {*},"Sydney, Australia",-33.865423,151.207317
137,Woollahra - Double Bay,"Sydney, Australia",-33.880873,151.243412
138,Bondi,"Sydney, Australia",-33.893056,151.263333
139,Surry Hills - Darlinghurst,"Sydney, Australia",-33.880456,151.216719
140,Elizabeth Bay House,"Sydney, Australia",-33.870085,151.226444
141,Randwick,"Sydney, Australia",-33.914121,151.241005
142,Rozelle,"Sydney, Australia",-33.864500,151.174354
143,Waterloo,"Sydney, Australia",-33.900276,151.207314
144,Chippendale - Ultimo,"Sydney, Australia",-33.884304,151.200078
145,Clovelly,"Sydney, Australia",-33.912639,151.261790


In [78]:
def getNearbyVenues(b_names, cities, latitudes, longitudes):
    
    radius=1000
    limit=100    
    
    venues_list=[]
    for b_name, city, lat, lng in zip(b_names, cities, latitudes, longitudes):
        print(b_name, end=' ')
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            b_name,
            city,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 'City+', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [79]:
# FETCH VENUES
venues = getNearbyVenues(b_names = hoods_ci['Borough'], 
                         cities = hoods_ci['City+'],
                         latitudes = hoods_ci['Latitude'], 
                         longitudes = hoods_ci['Longitude'])
print('\n''Done')

City {*} Woollahra - Double Bay Bondi Surry Hills - Darlinghurst Elizabeth Bay House Randwick Rozelle Waterloo Chippendale - Ultimo Clovelly Forest Lodge Lavender Bay Pyrmont Macdonaldtown 
Done


In [80]:
venues.shape

(1268, 8)

In [81]:
venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1268 entries, 0 to 1267
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Borough            1268 non-null   object 
 1   City+              1268 non-null   object 
 2   Borough Latitude   1268 non-null   float64
 3   Borough Longitude  1268 non-null   float64
 4   Venue              1268 non-null   object 
 5   Venue Latitude     1268 non-null   float64
 6   Venue Longitude    1268 non-null   float64
 7   Venue Category     1268 non-null   object 
dtypes: float64(4), object(4)
memory usage: 59.5+ KB


In [82]:
venues.tail()

,Borough,City+,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1263,Macdonaldtown,"Sydney, Australia",-33.896783,151.186337,The Grandstand Sports Bar & Function Centre,-33.888584,151.184630,Sports Bar
1264,Macdonaldtown,"Sydney, Australia",-33.896783,151.186337,Made 590,-33.904055,151.180101,Clothing Store
1265,Macdonaldtown,"Sydney, Australia",-33.896783,151.186337,Domino's Pizza,-33.892520,151.187360,Pizza Place
1266,Macdonaldtown,"Sydney, Australia",-33.896783,151.186337,Hoochie Mamma Cafe,-33.892144,151.182999,Café
1267,Macdonaldtown,"Sydney, Australia",-33.896783,151.186337,Buon Gusto,-33.892610,151.192010,Italian Restaurant


In [83]:
# how many boroughs returned
print('Number of boroughs returned is {} out of the original {}'\
      .format(len(venues['Borough'].unique()), len(hoods_ci)))

Number of boroughs returned is 14 out of the original 14


In [84]:
# how many unique venue categories
print('There are {} uniques venue categories'\
      .format(len(venues['Venue Category'].unique())))

There are 179 uniques venue categories


In [85]:
# list of venue categories
pd.set_option('display.max_rows', None)
venues_catgs = pd.Series(sorted(list(venues['Venue Category'].unique()))).to_frame()
venues_catgs.head(10)
# venues_catgs.to_csv('venues_catgs.csv')

,0
0,American Restaurant
1,Aquarium
2,Argentinian Restaurant
3,Art Gallery
4,Art Museum
5,Asian Restaurant
6,Australian Restaurant
7,Austrian Restaurant
8,Auto Workshop
9,BBQ Joint


In [86]:
# top 10 most frequent venues
top10 = venues['Venue Category'].value_counts().to_frame().head(10)
top10.index.rename('Venue Category', inplace=True)
top10.rename(columns={'Venue Category': 'Counts'}, inplace=True)
top10

,Counts
Venue Category,
Café,229
Bar,56
Park,48
Thai Restaurant,46
Pub,44
Coffee Shop,43
Italian Restaurant,42
Pizza Place,34
Bakery,34


In [87]:
# how many venues returned for each borough
venues.groupby(['Borough','City+'])['Venue'].count().to_frame() #.head(10) #.transpose()

,,Venue
Borough,City+,
Bondi,"Sydney, Australia",94
Chippendale - Ultimo,"Sydney, Australia",100
City {*},"Sydney, Australia",100
Clovelly,"Sydney, Australia",48
Elizabeth Bay House,"Sydney, Australia",100
Forest Lodge,"Sydney, Australia",100
Lavender Bay,"Sydney, Australia",100
Macdonaldtown,"Sydney, Australia",100
Pyrmont,"Sydney, Australia",100


In [48]:
# count venue categories for a given borough
boro = 'Stevens' #Kioicho' #'Dubai Marina' #'Telegraph Hill' #'Al Hisn'  #'Duomo' #'Battersea' #'Roppongi Station' #'Somerset'  #'Chippendale' #'Chūō'  #'Westminster'  
venues_boro = venues.loc[ venues['Borough']==boro ]\
    .groupby(['Venue Category'])['Venue Category']\
    .count().sort_values(ascending=False).to_frame()

venues_boro.rename(columns={'Venue Category': 'Counts'}, inplace=True)

venues_boro.head(10)

,Counts
Venue Category,
Hotel,6
Italian Restaurant,2
Garden,2
Gas Station,1
Botanical Garden,1
Bowling Alley,1
Buffet,1
Bus Station,1
Café,1


In [49]:
# find which boroughs have a given venue category
v_ctg = 'Arcade'  #''  #'Trail'
venues.loc[ venues['Venue Category']==v_ctg ]

,Borough,City+,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [88]:
# find similar venue categories by keyword + wildcard
v_ctg = '*Restaurant*'   #'*Bar*'  #'*Market*'  #'*Shop*'  #'*Store*'
matching = fnmatch.filter(venues['Venue Category'], v_ctg)
np.sort(pd.Series(matching).unique())[:50]

array(['American Restaurant', 'Argentinian Restaurant',
       'Asian Restaurant', 'Australian Restaurant', 'Austrian Restaurant',
       'Brazilian Restaurant', 'Cantonese Restaurant',
       'Chinese Restaurant', 'Dumpling Restaurant',
       'Fast Food Restaurant', 'French Restaurant', 'Greek Restaurant',
       'Hungarian Restaurant', 'Indian Restaurant',
       'Indonesian Restaurant', 'Italian Restaurant',
       'Japanese Restaurant', 'Kebab Restaurant', 'Korean BBQ Restaurant',
       'Korean Restaurant', 'Latin American Restaurant',
       'Lebanese Restaurant', 'Malay Restaurant',
       'Mediterranean Restaurant', 'Mexican Restaurant',
       'Middle Eastern Restaurant', 'Moroccan Restaurant',
       'Polish Restaurant', 'Portuguese Restaurant', 'Ramen Restaurant',
       'Restaurant', 'Seafood Restaurant', 'South American Restaurant',
       'South Indian Restaurant', 'Spanish Restaurant',
       'Sri Lankan Restaurant', 'Sushi Restaurant', 'Szechuan Restaurant',
       'Ta

temp file csv to append data

In [89]:
# first save (and create file)
# venues.to_csv('boroughs_list_with_coords_and_venues - SYDNEY DO-OVER.csv',index=False)

In [38]:
# check file before append
# pd.read_csv('boroughs_list_with_coords_and_venues.csv').tail()

In [39]:
# subsequent saves --> append mode
# venues.to_csv('boroughs_list_with_coords_and_venues.csv', mode='a', index=False , header=None)

In [41]:
# check file after append
# pd.read_csv('boroughs_list_with_coords_and_venues.csv').tail()

In [266]:
#
#
## save all data also as excel ## DO NOT USE FOR ONE CITY ONLY --> OVERRIDE!!
# venues.to_excel('boroughs_list_with_coords_and_venues.xlsx')
#
#

### Checking all venues from backup file

In [12]:
venz = pd.read_csv('boroughs_list_with_coords_and_venues.csv')
# venuz.head()
venz.groupby(['City+','Borough','Venue Category'])['Venue'].count().to_frame().head(10)
        # .sort_values(by='Venue', ascending=False)

Venue
City+                       Borough         Venue Category            
Dubai, United Arab Emirates Al Barsha 1 {*} Accessories Store        1
                                            American Restaurant      2
                                            Arcade                   1
                                            Asian Restaurant         2
                                            Athletics & Sports       1
                                            Bakery                   2
                                            Bed & Breakfast          1
                                            Boutique                 3
                                            Breakfast Spot           2
                                            Buffet                   1

In [6]:
# top 10 most frequent venues per city
venz = pd.read_csv('boroughs_list_with_coords_and_venues.csv')
# venz.rename(columns={'Borough':'Area'}, inplace=True)
venz['City+'] = venz['City+'].apply( lambda x: x.split(',')[0] )
venz.rename(columns={'City+':'City'}, inplace=True)

c_uniq = sorted(list(venz['City'].unique()))
top10pc = pd.DataFrame() 
for i in c_uniq:  # range(len(c_uniq)):
    c_iter =  pd.Series( venz.pivot_table( index='Venue Category', columns='City',aggfunc='count' )['Venue'].\
                  sort_values(by=i,ascending=False).head(10).index )
    top10pc[i] = c_iter

# top10pc.columns=c_uniq  # needed?
print('Most frequent venue categories per city')
top10pc.index = top10pc.index+1
top10pc

Most frequent venue categories per city


,Dubai,London,Milan,New York,Paris,San Francisco,Singapore,Sydney,Tokyo,Zurich
1,Hotel,Pub,Italian Restaurant,Coffee Shop,French Restaurant,Coffee Shop,Hotel,Café,Japanese Restaurant,Italian Restaurant
2,Café,Coffee Shop,Pizza Place,Italian Restaurant,Hotel,Park,Chinese Restaurant,Bar,Ramen Restaurant,Swiss Restaurant
3,Coffee Shop,Café,Café,Park,Italian Restaurant,Bakery,Coffee Shop,Park,Café,Café
4,Middle Eastern Restaurant,Hotel,Ice Cream Shop,American Restaurant,Bakery,Café,Café,Thai Restaurant,Coffee Shop,Hotel
5,Indian Restaurant,Italian Restaurant,Hotel,Bakery,Coffee Shop,Pizza Place,Japanese Restaurant,Pub,Sake Bar,Restaurant
6,Restaurant,Bakery,Plaza,Café,Plaza,Chinese Restaurant,Food Court,Coffee Shop,BBQ Joint,Bar
7,Asian Restaurant,Park,Japanese Restaurant,Mexican Restaurant,Bar,Sushi Restaurant,Bakery,Italian Restaurant,Italian Restaurant,Supermarket
8,Burger Joint,Gym / Fitness Center,Cocktail Bar,Gym,Japanese Restaurant,Wine Bar,Noodle House,Pizza Place,Chinese Restaurant,Bakery
9,Fast Food Restaurant,Theater,Restaurant,Pizza Place,Vietnamese Restaurant,Mexican Restaurant,Indian Restaurant,Bakery,Soba Restaurant,Tram Station
10,Gym / Fitness Center,French Restaurant,Seafood Restaurant,Grocery Store,Café,Vietnamese Restaurant,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Park


In [35]:
c='Zurich'
list(venz.pivot_table( index='Venue Category', columns='City', aggfunc='count' )['Venue'] .\
    sort_values(by=c,ascending=False).head(10).index)

['Italian Restaurant',
 'Swiss Restaurant',
 'Café',
 'Hotel',
 'Restaurant',
 'Bar',
 'Supermarket',
 'Bakery',
 'Tram Station',
 'Park']

In [37]:
c_uniq = list(venz['City'].unique())
c_uniq #[0]

['Zurich',
 'Milan',
 'New York',
 'Dubai',
 'Tokyo',
 'London',
 'Singapore',
 'San Francisco',
 'Paris',
 'Sydney']

#### [back to contents](#contentzs)

----
----

#### Visualize **map** of venues in one city <a name="map-ven"></a>

In [7]:
# if fetching from file:
venues = pd.read_csv('boroughs_list_with_coords_and_venues.csv')
    # pd.read_excel('boroughs_list_with_coords_and_venues.xlsx')
# venues.head()
# venues.tail()

v = list( venues['City+'].unique() )
for i,n in enumerate(v):
    print(i,n)  
# v[1]

0 Zurich, Switzerland
1 Milan, Italy
2 New York, NY, United States
3 Dubai, United Arab Emirates
4 Tokyo, Japan
5 London, United Kingdom
6 Singapore, Singapore
7 San Francisco, CA, United States
8 Paris, France
9 Sydney, Australia


In [17]:
cx = v[0]  # enter city index number 
venues_cx = venues.loc[venues['City+']==cx ]  # venues['City+']==cx ]    ##venues['Borough']=='Chelsea-Knightsbridge'
venues_cx.tail() #head()

,Borough,City+,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
830,Doldertal,"Zurich, Switzerland",47.372722,8.564521,VBZ Klusplatz,47.364090,8.566398,Tram Station
831,Doldertal,"Zurich, Switzerland",47.372722,8.564521,Kluspark,47.364476,8.564989,Park
832,Doldertal,"Zurich, Switzerland",47.372722,8.564521,VBZ Platte,47.375031,8.552778,Tram Station
833,Doldertal,"Zurich, Switzerland",47.372722,8.564521,VBZ Voltastrasse,47.377029,8.553893,Tram Station
834,Doldertal,"Zurich, Switzerland",47.372722,8.564521,Moser's Backparadies,47.364048,8.566491,Bakery


In [18]:
%%time

c0 = cx
address_0 = c0 ; print(c0)
geolocator = Nominatim(user_agent="explorateur")
location_0 =  geolocator.geocode(address_0)
latitude_0 = location_0.latitude
longitude_0 = location_0.longitude

map_venues = folium.Map(location=[latitude_0, longitude_0], zoom_start=12)
                       #, tiles='Stamen Terrain') 


mc = MarkerCluster() #.add_to(map_venues)

# loop for venues
for ven, v_cat, v_lat, v_lon in zip(venues_cx['Venue'], venues_cx['Venue Category'],
                                venues_cx['Venue Latitude'], venues_cx['Venue Longitude']):  
                                #vbor #venues_cx['Borough']
       
    v_label = '{}, {}'.format(ven[:15], v_cat) #, vbor)
    v_label = folium.Popup(v_label, parse_html=True)
    folium.CircleMarker(  
                [v_lat, v_lon],
                radius=5,
                popup=v_label,
                color='black',
                weight=2,
                fill=True,
                fill_color='red', #yellow',  #'#3186cc',
                fill_opacity=0.7,
                parse_html=False
                ).add_to(mc)  # add_to(map_venues)  ##marker_cluster

mc.add_to(map_venues)
folium.LayerControl().add_to(map_venues)

# loop for boroughs
for b_lat, b_lon, bor in zip(venues_cx['Borough Latitude'], venues_cx['Borough Longitude'],\
                            venues_cx['Borough']):
    b_label = '{}'.format(bor)
    b_label = folium.Popup(b_label, parse_html=True)
    folium.Circle(   #CircleMarker  ## map_venues.add_child(
                [b_lat, b_lon],
                radius=1000,  # for Circle this is radius in meters!
                popup=b_label,
                color='black',
                weight=2,
                fill=True,
                fill_color='red',  #'yellow',  #'#3186cc',
                fill_opacity=0.0023,
                parse_html=False
                ).add_to(map_venues)

# visualize all
map_venues

Zurich, Switzerland
Wall time: 22.6 s


In [19]:
# save map to file
# map_venues.save(outfile= "venues_map_ZURICH_4.html")

In [77]:
# does not work for maps
# map_venues.savefig('venues_map_test.png')

In [75]:
# does not work 
# import io
# from PIL import Image 
# img_data = map_venues._to_png(5)
# img = Image.open(io.BytesIO(img_data))
# img.save('venues_map_test.png')

#### [back to contents](#contentzs)

#### [back to contents](#contentzs)

#### [back to contents](#contentzs)